# Connecting to Fatboy  May 4th 2015

Fatboy is a Windows machine sitting on the 2nd floor of the PAB. It runs Microsoft SQL Server 2008, which currently holds the primary 20TB CATSIM databases.

There are currently two separate and distinct problems: 
    (1) network access (private vs. private ip, port, firewalls) and
    (2) sql server authentication from the stack.
 
They are both being addressed a the same time because the improved sql server authentication also makes it simpler to connect to the database through an ssh tunnel.  

## Problem 1: Network Access

Fatboy had been on a private IP address (172...) until a year ago. Up until that point, it was only accessible on the PAB wired network or through an ssh tunnel.  We moved it to a public IP address when collaborators without UW NETIDs wanted access. The protocol for this past year was to send us the source ip address which got added to the whitelist on a windows firewall. This was terrible for all people and machines involved.

Problems:
  * The windows firewall can't keep up with the incoming traffic. 6% of incoming connection attempts time out. 
  * Source ip addresses change. It was annoying to only be able to access from some subnets.  
  * Some institutions block port 1433.
  
**We are returning to a standard authentication method: ssh. **
See "Demo: Off Campus" section below. 

But first, let's configure our authentication utility:

## Problem 2: SQL SERVER Authentication

This problem is unrelated to the network access. The code in sim_catUtils included some hard coded connection strings. (Google fatboy.npl.washington.edu, and you will see our passwords in plain text). This account does not have write access, which is why our databases haven't been deleted yet. 

The DM stack includes a utility for hiding passwords in [daf.persistence.dbAuth](https://lsst-web.ncsa.illinois.edu/~buildbot/doxygen/x_masterDoxyDoc/classlsst_1_1daf_1_1persistence_1_1_db_auth.html)

After [tickets/SIM-1092](https://jira.lsstcorp.org/browse/SIM-1092) is merged you will need to add two configuration files to your home directory.  **dbAuth** will look for these files to retrieve your username and password for your databases. 


First, Let's come up with a new password.


Now, you must create **two files**  under your home directory, in a sub-directory called <code> $HOME/.lsst</code>.


#### 1) <code> $HOME/.lsst/db-auth.paf </code>


The first is a DB authorization file to present login credentials to whatever DB server is being used. To do this, create a <code>$HOME/.lsst/db-auth.paf</code> file with the following content: 

<code>
database: {
    authInfo: {
        host: fatboy.npl.washington.edu
        port: 1433
        user: LSST-2
        password: [Our secret Password]
    }

    authInfo: {    
        host: fatboy.npl.washington.edu
        port: 3333
        user: LSST-2
        password: [Our secret Password]
    }
    authInfo: {
        host: localhost
        port: 5555
        user: LSST-2
        password: [Our secret Password]
    }
}
</code>
    
Authorization information for more than one DB server may be included.

#### 2)  <code> $HOME/.lsst/db-auth.py </code>


The config file: <code> $HOME/.lsst/db-auth.py </code> has content that is almost identical to the above policy file: 

The directory <code>$HOME/.lsst</code> must have 700 permissions and these files must each have 600 permissions (go-rwx in all cases). 

<code>
$> cd $HOME
$> chmod 700 .lsst
$> chmod 600 .lsst/db-auth.py 
$> chmod 600 .lsst/db-auth.paf
</code>

# CATSIM Demo

### On Campus

In [9]:
from lsst.sims.catUtils.baseCatalogModels import GalaxyAgnObj
from lsst.sims.utils import ObservationMetaData

obsMetadata = ObservationMetaData(unrefractedRA=45., unrefractedDec=1.0,
                                          boundType='circle', boundLength=0.05,
                                          mjd=57086)
galaxyAgnDB = GalaxyAgnObj(verbose=True)
#result = galaxyAgnDB.query_columns(['raJ2000', 'decJ2000'], 
#                                   obs_metadata=obsMetadata)


2015-05-04 12:21:36,175 INFO sqlalchemy.engine.base.Engine select @@version


INFO:sqlalchemy.engine.base.Engine:select @@version


2015-05-04 12:21:36,176 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2015-05-04 12:21:36,227 INFO sqlalchemy.engine.base.Engine 
            SELECT default_schema_name FROM
            sys.database_principals
            WHERE principal_id=database_principal_id()
        


...

INFO:sqlalchemy.engine.base.Engine:{'tabname': 'galaxy', 'schname': u'dbo'}


2015-05-04 12:21:37,821 INFO sqlalchemy.engine.base.Engine select ind_col.index_id, ind_col.object_id, col.name from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=%(tabname)s and sch.name=%(schname)s


INFO:sqlalchemy.engine.base.Engine:select ind_col.index_id, ind_col.object_id, col.name from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=%(tabname)s and sch.name=%(schname)s


2015-05-04 12:21:37,822 INFO sqlalchemy.engine.base.Engine {'tabname': 'galaxy', 'schname': u'dbo'}


INFO:sqlalchemy.engine.base.Engine:{'tabname': 'galaxy', 'schname': u'dbo'}
Database column, htmid, overridden in self.columns... Skipping default assignment.
Database column, galid, overridden in self.columns... Skipping default assignment.
Database column, ra, overridden in self.columns... Skipping default assignment.
Database column, dec, overridden in self.columns... Skipping default assignment.
Database column, redshift, overridden in self.columns... Skipping default assignment.
Database column, rad_vel, overridden in self.columns... Skipping default assignment.
Database column, u_ab, overridden in self.columns... Skipping default assignment.
Database column, g_ab, overridden in self.columns... Skipping default assignment.
Database column, r_ab, overridden in self.columns... Skipping default assignment.
Database column, i_ab, overridden in self.columns... Skipping default assignment.
Database column, z_ab, overridden in self.columns... Skipping default assignment.
Database column,

### Off Campus

First you need to establish your ssh tunnel, using local port forwarding switch -L. 

For example: 
<code>
$>  ssh -L 5555:fatboy.npl.washington.edu:1433 yusra@gateway.astro.washington.edu
</code>

Your local ssh client will connect to the ssh server running on gateway.astro.washington.edu and bind the local port 1434 to remote port 1433 of fatboy.npl.washington.edu.  
gateway.astro.washington.edu is within the private network and reach the fatboy. 

Now from your local computer, the database looks like it is at localhost:5555

In [11]:
from lsst.sims.catUtils.baseCatalogModels import GalaxyAgnObj
from lsst.sims.utils import ObservationMetaData

obsMetadata = ObservationMetaData(unrefractedRA=45., unrefractedDec=1.0,
                                          boundType='circle', boundLength=0.05,
                                          mjd=57086)

galaxyAgnDB = GalaxyAgnObj(verbose=True, host='localhost', port='5555')

2015-05-04 12:22:19,199 INFO sqlalchemy.engine.base.Engine select @@version


INFO:sqlalchemy.engine.base.Engine:select @@version


2015-05-04 12:22:19,202 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2015-05-04 12:22:19,252 INFO sqlalchemy.engine.base.Engine 
            SELECT default_schema_name FROM
            sys.database_principals
            WHERE principal_id=database_principal_id()
        


...

INFO:sqlalchemy.engine.base.Engine:{'tabname': 'galaxy', 'schname': u'dbo'}


2015-05-04 12:22:20,279 INFO sqlalchemy.engine.base.Engine select ind_col.index_id, ind_col.object_id, col.name from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=%(tabname)s and sch.name=%(schname)s


INFO:sqlalchemy.engine.base.Engine:select ind_col.index_id, ind_col.object_id, col.name from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=%(tabname)s and sch.name=%(schname)s


2015-05-04 12:22:20,282 INFO sqlalchemy.engine.base.Engine {'tabname': 'galaxy', 'schname': u'dbo'}


INFO:sqlalchemy.engine.base.Engine:{'tabname': 'galaxy', 'schname': u'dbo'}
Database column, htmid, overridden in self.columns... Skipping default assignment.
Database column, galid, overridden in self.columns... Skipping default assignment.
Database column, ra, overridden in self.columns... Skipping default assignment.
Database column, dec, overridden in self.columns... Skipping default assignment.
Database column, redshift, overridden in self.columns... Skipping default assignment.
Database column, rad_vel, overridden in self.columns... Skipping default assignment.
Database column, u_ab, overridden in self.columns... Skipping default assignment.
Database column, g_ab, overridden in self.columns... Skipping default assignment.
Database column, r_ab, overridden in self.columns... Skipping default assignment.
Database column, i_ab, overridden in self.columns... Skipping default assignment.
Database column, z_ab, overridden in self.columns... Skipping default assignment.
Database column,

### Changing the default database in sims_catUtils

in tickets/SIM-1092
edit the file: $SIMS_CATUTILS_DIR/policy/db.paf

after change:

Now you can run demo again without specifying the host and port when instantiating.


In [12]:
galaxyAgnDB = GalaxyAgnObj(verbose=True)

2015-05-04 12:22:32,863 INFO sqlalchemy.engine.base.Engine select @@version


INFO:sqlalchemy.engine.base.Engine:select @@version


2015-05-04 12:22:32,865 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2015-05-04 12:22:32,914 INFO sqlalchemy.engine.base.Engine 
            SELECT default_schema_name FROM
            sys.database_principals
            WHERE principal_id=database_principal_id()
        


...

INFO:sqlalchemy.engine.base.Engine:{'tabname': 'galaxy', 'schname': u'dbo'}


2015-05-04 12:22:33,938 INFO sqlalchemy.engine.base.Engine select ind_col.index_id, ind_col.object_id, col.name from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=%(tabname)s and sch.name=%(schname)s


INFO:sqlalchemy.engine.base.Engine:select ind_col.index_id, ind_col.object_id, col.name from sys.columns as col join sys.tables as tab on tab.object_id=col.object_id join sys.index_columns as ind_col on (ind_col.column_id=col.column_id and ind_col.object_id=tab.object_id) join sys.schemas as sch on sch.schema_id=tab.schema_id where tab.name=%(tabname)s and sch.name=%(schname)s


2015-05-04 12:22:33,939 INFO sqlalchemy.engine.base.Engine {'tabname': 'galaxy', 'schname': u'dbo'}


INFO:sqlalchemy.engine.base.Engine:{'tabname': 'galaxy', 'schname': u'dbo'}
Database column, htmid, overridden in self.columns... Skipping default assignment.
Database column, galid, overridden in self.columns... Skipping default assignment.
Database column, ra, overridden in self.columns... Skipping default assignment.
Database column, dec, overridden in self.columns... Skipping default assignment.
Database column, redshift, overridden in self.columns... Skipping default assignment.
Database column, rad_vel, overridden in self.columns... Skipping default assignment.
Database column, u_ab, overridden in self.columns... Skipping default assignment.
Database column, g_ab, overridden in self.columns... Skipping default assignment.
Database column, r_ab, overridden in self.columns... Skipping default assignment.
Database column, i_ab, overridden in self.columns... Skipping default assignment.
Database column, z_ab, overridden in self.columns... Skipping default assignment.
Database column,

## What if I don't have a UW NetId?

We are now maintaining a shared UW NETID "**simsuser**" for our collaborators port forward using gateway.astro.washington.edu. The preferred protocol now to send us your public ssh key. 

Once we add it, You can now access our database from *anywhere* like:

# Going Forward

* Simon had the interesting idea of maintaining a lsst github repo and using a deployment service to update the ssh keys in simsusers home directory. People already have to ask Frossie/Michael for push permissions to our lsst repos. 



* We can initialize the ssh tunnel through python in sims_catUtils.
    - add new third party package paramiko?
    - write our own utility?
